In [1]:
import optlang
import numpy as np
import sympy

from optlang import Model, Variable, Constraint, Objective

# All the (symbolic) variables are declared, with a name and optionally a lower and/or upper bound.
x1 = Variable('x1', lb=5, ub=100, type="integer")
x2 = Variable('x2', lb=0, ub=100, type="integer")
x3 = Variable('x3', lb=0, ub=100, type="integer")

# A constraint is constructed from an expression of variables and a lower and/or upper bound (lb and ub).
c1 = Constraint(x1 + x2 + x3, ub=100)
c2 = Constraint(10 * x1 + 4 * x2 + 5 * x3, ub=600)
c3 = Constraint(2 * x1 + 2 * x2 + 6 * x3, ub=300)

# An objective can be formulated
obj = Objective(10 * x1 + 6 * x2 + 4 * x3, direction='max')

# Variables, constraints and objective are combined in a Model object, which can subsequently be optimized.
prob = Model(name='Simple model')
prob.objective = obj
prob.add([c1, c2, c3])

status = prob.optimize()

opt_sol = {}
print("status:", prob.status)
print("objective value:", prob.objective.value)
print("----------")
for var_name, var in prob.variables.iteritems():
    print(var_name, "=", var.primal)
    opt_sol[var] = var.primal

status: optimal
objective value: 732.0
----------
x1 = 33.0
x2 = 67.0
x3 = 0.0


In [14]:
import sys
sys.path.append("..")
from neurop.Problem import ILPProblem
from neurop.Model import QUBOModel
from neurop.Backend import SimulatedAnnealingBackend

backend = SimulatedAnnealingBackend(np.logspace(5, -1, 100_000))
problem = ILPProblem(prob, penalty=100)
model = problem.convert_to_model(QUBOModel, backend)
#model.Q

In [15]:
sol_energy, sol_assignment = model.run()
problem_params = model.to_problem_parameters(sol_assignment)
problem_params

  0%|          | 0/100000 [00:00<?, ?it/s]

Temperature: 0.1  Current energy: -26675740: 100%|██████████| 100000/100000 [00:04<00:00, 20623.94it/s]               


{5 <= x1 <= 100: 76, 0 <= x2 <= 100: 4, 0 <= x3 <= 100: 20}

In [16]:
problem.evaluate_objective(problem_params)

864.000000000000

In [18]:
problem.evaluate_constraints(problem_params)

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True, False],
       [ True,  True]])

In [6]:
sympy.Matrix(model.Q)

Matrix([
[1057790,    21200,    42400,    84800,   169600,    339200,    678400,    4500,    9000,   18000,    36000,    72000,   144000,    288000,    6300,   12600,    25200,    50400,   100800,   201600,    403200,   -200,   -400,    -800,   -1600,   -3200,    -6400,   -12800,   -25600,   -51200,  -1000,   -2000,   -4000,   -8000,   -16000,   -32000,   -64000,   -128000,   -256000,   -512000,  -100,   -200,   -400,   -800,   -1600,   -3200,   -6400,     0,      0,      0,      0,       0,       0,       0,     0,      0,      0,      0,       0,       0,       0,  -100,   -200,   -400,   -800,   -1600,   -3200,   -6400],
[  21200,  2136780,    84800,   169600,   339200,    678400,   1356800,    9000,   18000,   36000,    72000,   144000,   288000,    576000,   12600,   25200,    50400,   100800,   201600,   403200,    806400,   -400,   -800,   -1600,   -3200,   -6400,   -12800,   -25600,   -51200,  -102400,  -2000,   -4000,   -8000,  -16000,   -32000,   -64000,  -128000,   -256000, 

In [7]:
var_a = sympy.Array(model.variables, (len(model.variables),1))
(var_a.transpose() @ sympy.Matrix(model.Q) @ var_a)[0].expand()

-44100*s_0_0**2 + 400*s_0_0*s_0_1 + 800*s_0_0*s_0_2 + 1600*s_0_0*s_0_3 + 3200*s_0_0*s_0_4 + 6400*s_0_0*s_0_5 + 12800*s_0_0*s_0_6 + 25600*s_0_0*s_0_7 + 51200*s_0_0*s_0_8 - 400*s_0_0*x1_0 - 800*s_0_0*x1_1 - 1600*s_0_0*x1_2 - 3200*s_0_0*x1_3 - 6400*s_0_0*x1_4 - 12800*s_0_0*x1_5 - 25600*s_0_0*x1_6 - 400*s_0_0*x2_0 - 800*s_0_0*x2_1 - 1600*s_0_0*x2_2 - 3200*s_0_0*x2_3 - 6400*s_0_0*x2_4 - 12800*s_0_0*x2_5 - 25600*s_0_0*x2_6 - 1200*s_0_0*x3_0 - 2400*s_0_0*x3_1 - 4800*s_0_0*x3_2 - 9600*s_0_0*x3_3 - 19200*s_0_0*x3_4 - 38400*s_0_0*x3_5 - 76800*s_0_0*x3_6 - 88000*s_0_1**2 + 1600*s_0_1*s_0_2 + 3200*s_0_1*s_0_3 + 6400*s_0_1*s_0_4 + 12800*s_0_1*s_0_5 + 25600*s_0_1*s_0_6 + 51200*s_0_1*s_0_7 + 102400*s_0_1*s_0_8 - 800*s_0_1*x1_0 - 1600*s_0_1*x1_1 - 3200*s_0_1*x1_2 - 6400*s_0_1*x1_3 - 12800*s_0_1*x1_4 - 25600*s_0_1*x1_5 - 51200*s_0_1*x1_6 - 800*s_0_1*x2_0 - 1600*s_0_1*x2_1 - 3200*s_0_1*x2_2 - 6400*s_0_1*x2_3 - 12800*s_0_1*x2_4 - 25600*s_0_1*x2_5 - 51200*s_0_1*x2_6 - 2400*s_0_1*x3_0 - 4800*s_0_1*x3_1 - 9

In [8]:
import neal
import dimod

bqm=dimod.BinaryQuadraticModel(np.array(model.Q, dtype=int), "BINARY")
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, beta_schedule_type="geometric", num_reads=1000, num_sweeps=1000)
sampleset.relabel_variables(dict(enumerate(model.variables)))
#decoded_samples = model.decode_sampleset(sampleset)
#best_sample = min(decoded_samples, key=lambda x: x.energy)
#pprint(best_sample.sample)
sol_assignment_neal=sampleset.lowest().first.sample

problem_params_neal = model.to_problem_parameters(sol_assignment_neal)
problem_params_neal

{5 <= x1 <= 100: 38, 0 <= x2 <= 100: 46, 0 <= x3 <= 100: 2}

In [9]:
problem.evaluate_objective(problem_params_neal)

664.000000000000

In [19]:
print("==== Results ====")
for k in problem.problem.variables:
    print("{}: ours: {} neal: {} optimal: {}".format(k.name, problem_params[k], problem_params_neal[k], opt_sol[k]))

==== Results ====
x1: ours: 76 neal: 38 optimal: 33.0
x2: ours: 4 neal: 46 optimal: 67.0
x3: ours: 20 neal: 2 optimal: 0.0
